In [2]:
import pandas as pd
import numpy as np

Read in the data:

In [3]:
DATA_RAW = pd.read_csv('../source data/new data/Presented_Clean_V6_Merged.csv')
DATA_ENG_WAL = DATA_RAW[(DATA_RAW['ONS_CENSUS_2011'].str[0] == 'E') | (DATA_RAW['ONS_CENSUS_2011'].str[0] == 'W')]
data_train = DATA_ENG_WAL[DATA_ENG_WAL['LA_POPULATION'].notnull()]
#data_eng_nonan = data_eng_nonan[data_eng_nonan['Modifed_Total'].notnull()]

The simplest, dumbest approximation: calculate the average proportion of "Total presented persons" to "Local authority population".

In [4]:
avg_homeless = (data_train['Modifed_Total']/data_train['LA_POPULATION']).mean()
print(avg_homeless)

0.00134531046955


"Predict" the number of _presented_ persons as the (national) average homeless rate times local authority population: (`notnull()` is there because we are only interested in existing data, as we want to compare our prediction with them)

In [5]:
prediction = avg_homeless*data_train[data_train['Modifed_Total'].notnull()]['LA_POPULATION']

In [6]:
from sklearn.metrics import r2_score, mean_absolute_error, median_absolute_error

def print_metrics(target, prediction):
    print('R^2 = {0}'.format(r2_score(target, prediction)))
    print('Mean error = {0}'.format(mean_absolute_error(target, prediction)))
    print('Median error = {0}'.format(median_absolute_error(target, prediction)))
    
print_metrics(data_train['Modifed_Total'].dropna(), prediction)

R^2 = 0.40505833102009337
Mean error = 156.4523060664268
Median error = 106.6597312591262


Now, apply this **very rough** approximation to all English and Welsh districts:

In [7]:
baseline_prediction = DATA_ENG_WAL.copy()
baseline_prediction['predicted_presented'] = avg_homeless*baseline_prediction['LA_POPULATION']

In [8]:
baseline_prediction.to_csv('baseline_prediction.csv', columns=['ONS_CENSUS_2011', 'Local.Authority', 'predicted_presented'])

And the "predicted" number of homeless young people is...

In [10]:
print(baseline_prediction['predicted_presented'].sum())

75686.55221
